<a href="https://colab.research.google.com/github/chiffonng/mnemonic-gen/blob/sft-re/notebooks/sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_wJvZr5c8mi-59PSsDlPszkgD2hUIo0t?usp=sharing&authuser=1)

In [1]:
#@title Install dependencies
%%capture

!uv pip cache purge

!uv pip install --system unsloth wandb ray # bitsandbytes triton
# Get the latest nightly Unsloth
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch

if torch.cuda.get_device_capability()[0] >= 8:
    !uv pip install --system --no-deps packaging ninja einops "flash-attn>=2.6.3"

### Get current GPU and memory stats

In [2]:
#!nvidia-smi

In [3]:
import psutil

# Get the memory usage
memory_info = psutil.virtual_memory()
print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Memory Usage: {memory_info.percent}%")

Total Memory: 12.67 GB
Available Memory: 10.98 GB
Used Memory: 1.39 GB
Memory Usage: 13.4%


### Load model

In [4]:
from unsloth import FastLanguageModel

model_name = "unsloth/gemma-2-9b-it-bnb-4bit"

max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    #local_files_only=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Gemma2 patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

### Inference


In [5]:
import sys

from huggingface_hub import login


def is_colab():
    return "google.colab" in sys.modules


if is_colab():
    from google.colab import userdata

    HF_TOKEN = userdata.get("HF_TOKEN")
    WB_API_KEY = userdata.get("WB_API_KEY")

    if HF_TOKEN is None:
        raise KeyError("HF_TOKEN not found in Google Colab userdata.")

else:
    from dotenv import load_dotenv
    import os

    load_dotenv()

    try:
        HF_TOKEN = os.getenv("HF_TOKEN")
        WB_API_KEY = os.getenv("WB_API_KEY")
    except KeyError:
        raise KeyError("HF_TOKEN not found in environment variables.")


login(token=HF_TOKEN, add_to_git_credential=True, write_permission=False)
print("Login to HF hub successfully!")

Login to HF hub successfully!


<a name="Data"></a>

### Data Prep

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!


In [11]:
from datasets import load_dataset

mnemonic_prompt = """
You are an expert English teacher and mnemonic creator. Your task is to generate creative, effective mnemonic devices that help students learn English vocabulary, covering both the meaning and the correct spelling of each term. For each target term, produce one or more mnemonic devices along with a relevant explanation of your reasoning. Follow these guidelines:

- Use the target term correctly in context. Ensure the mnemonic is original, error-free, and appropriate for learners of all ages and cultural backgrounds.
- If the term has multiple spellings (e.g., British vs. American) or meanings, ask for clarification. If the term is commonly misspelled, include a mnemonic to prevent the error.
- Provide clear reasoning for each mnemonic, and explain how it connects to the target term. Focus on linguistic features such as morphemes (prefix, root, suffix), sememes, phonology, etymology, keyword associations, or contextual clues. Avoid arbitrary reasoning, circular reasoning (using the target term to learn it), and acronyms.
- Keep both the mnemonic and its explanation clear, simple, and easy to recall.

### Term:
{}

### Mnemonic:
{}"""

EOS_TOKEN = tokenizer.eos_token  # End-of-sequence token


# Formatting function
def formatting_prompts_func(examples):
    terms = examples["term"]
    mnemonics = examples["mnemonic"]
    texts = []
    for term, mnemonic in zip(terms, mnemonics, strict=False):
        text = mnemonic_prompt.format(term, mnemonic) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

# Load and process the dataset
dataset = load_dataset(
    "chiffonng/en-vocab-mnemonics",
    split={"train": "train", "val": "val"},
).map(formatting_prompts_func, batched=True)

test_dataset = load_dataset(
    "chiffonng/en-vocab-mnemonics-test", split="test"
)

train_dataset = dataset["train"]
validation_dataset = dataset["val"]

<a name="Train"></a>

### Train the model (Supervised fine-tuning, SFT)

Use Huggingface TRL's `SFTTrainer` for supervised finetuning! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).


In [9]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.051 GB of memory reserved.


#### LoRA configuration

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Rank > 0. Suggested 8, 16, 32, 64, 128. Higher ranks can capture more complex patterns but may increase the risk of overfitting and require more computational resources.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,  # = alpha/\sqrt{r} is a scaling factor for rank-stabilized LoRA. Should be >= 1.
    lora_dropout=0,  # Slight dropout to prevent overfitting
    bias="none",  # Supports any, but = "none" is optimized
    # "unsloth" uses 30% less VRAM, fits 2x larger batch sizes
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=True,  # Rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2025.3.9 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


TODO here:
- Hyperparameter tune
    - `lr_scheduler_type`: restart intervals and decay parameters

#### Logging

- HuggingFace transformers and wandb integration: [guide](https://docs.wandb.ai/guides/integrations/huggingface/)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [12]:
from transformers import EarlyStoppingCallback, TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

import wandb

def train_func():

    wandb.login(key=WB_API_KEY)
    wandb.init(project="gemma2-it-en-mnemonics-basic")

    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

    train_args = TrainingArguments(
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,  # batch size of 16
        warmup_steps=10,
        num_train_epochs=4,
        learning_rate=2e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine_with_restarts", # Cosine Annealing with Restarts
        seed=3407,

        # Save strategy
        output_dir="./outputs",
        save_strategy="steps",
        save_steps=10,
        load_best_model_at_end=True,
        save_total_limit=5,

        # Eval strategy
        metric_for_best_model="eval_loss",
        eval_strategy="steps",  # Enable evaluation during training
        eval_steps=10,  # Evaluate every 10 steps

        # Logging
        logging_steps=10,
        report_to="wandb",
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        dataset_text_field="text",

        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=True,
        args=train_args,
        callbacks=callbacks,
    )

    trainer_stats = trainer.train()
    wandb.finish()
    return trainer_stats

trainer_stats = train_func()

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/1047 [00:00<?, ? examples/s]

Packing train dataset (num_proc=2):   0%|          | 0/1047 [00:00<?, ? examples/s]

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/262 [00:00<?, ? examples/s]

Packing eval dataset (num_proc=2):   0%|          | 0/262 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 264 | Num Epochs = 4 | Total steps = 64
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 27,009,024/5,106,916,864 (0.53% trained)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chiffonng (chiffonng-minerva-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,0.915500,0.648066
40,0.304700,0.296067
60,0.275400,0.279024


Unsloth: Not an error, but Gemma2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


### Script to run

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

### Saving to float16 and push to hub.

Select `merged_16bit` for float16 or `merged_4bit` for int4. Use `push_to_hub_merged` to upload to Hugging Face account.


In [ ]:

model.push_to_hub_merged(
    "gemma2-9b-mnemonics-sft",
    tokenizer,
    save_method="merged_16bit",
    token=HF_TOKEN,
    safe_serialization=None # save to .safetensors
)


Unsloth: You are pushing to hub, but you passed your HF username = chiffonng.
We shall truncate chiffonng/gemma2-9b-mnemonics-sft to gemma2-9b-mnemonics-sft


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.96 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 38%|███▊      | 16/42 [02:19<03:58,  9.18s/it]